In [22]:
import numpy as np
import pandas as pd
from tqdm import tqdm

def add_basic_stats(row, moves, pref=""):

    row[pref + 'winrate_mean'] = np.mean(moves.winrate_delta)
    row[pref + 'score_mean'] = np.mean(moves.score_delta)
    row[pref + 'score_var'] = np.var(moves.score_delta)
    row[pref + 'selfplay_mean'] = np.mean(moves.selfplay_delta)
    row[pref + 'utility_mean'] = np.mean(moves.utility_delta)
    row[pref + 'winrate_beauty_percent'] = np.mean([x > 0 for x in moves.winrate_delta])
    row[pref + 'score_beauty_percent'] = np.mean([x > 0 for x in moves.score_delta])


def add_advanced_stats(row, moves, pref=""):
    moves.score_delta.sort()
    moves.winrate_delta.sort()

    row[pref + 'score25p'] = moves.score_delta[int(moves.cnt_moves * 0.25)]
    row[pref + 'score75p'] = moves.score_delta[int(moves.cnt_moves * 0.75)]
    row[pref + 'score_max'] = np.max(moves.score_delta)
    row[pref + 'score_min'] = np.min(moves.score_delta)
    row[pref + 'winrate25p'] = moves.winrate_delta[int(moves.cnt_moves * 0.25)]
    row[pref + 'winrate75p'] = moves.winrate_delta[int(moves.cnt_moves * 0.75)]

    row[pref + 'score_five_best_mean'] = np.mean(moves.score_delta[-5:])
    row[pref + 'score_five_worst_mean'] = np.mean(moves.score_delta[:5])

    row[pref + 'stddev_last'] = moves.stddev_delta[-1]
    moves.stddev_delta.sort()
    row[pref + 'stddev_mean'] = np.mean(moves.stddev_delta)
    row[pref + 'stddev50p'] = moves.stddev_delta[int(moves.cnt_moves * 0.5)]

    row[pref + 'score50p'] = moves.score_delta[int(moves.cnt_moves * 0.5)]

    row[pref + 'winrate_midmean'] = np.mean(
        moves.winrate_delta[int(moves.cnt_moves * 0.25):int(moves.cnt_moves * 0.75)])
    row[pref + 'score_midmean'] = np.mean(moves.score_delta[int(moves.cnt_moves * 0.25):int(moves.cnt_moves * 0.75)])


def get_index_dan(rank):
  for i in range(len(rank)):
    if rank[i] in ['k', 'd']:
      return i
  return -1


def get_int_from_rank(rank):
    ind = get_index_dan(rank)
    if ind < 0 or rank[0] == 'P' or not rank[:ind].isdigit():
        return None
    if rank[ind] == 'k':
        return -int(rank[:ind]) + 1
    else:
        return int(rank[:ind])


def get_rank_from_int(x):
    if x > 0:
        return str(x) + "d"
    else:
        return str(-x + 1) + "k"


def int_from_player(player):
    return int(player == 'W')


def player_from_int(x):
    return 'W' if x == 1 else 'B'


def add_meta(row, player='W'):
    if row['Result'] == '?':
        row['game_result'] = 0
    else:
        row['game_result'] = int(row['Result'])
        if player == 'B':
            row['game_result'] = -row['game_result']
    row['color'] = int_from_player(player)
    row['rank'] = get_int_from_rank(row[player + '_rating'])
    row['game_length'] = len(row['W_move'].split()) + len(row['B_move'].split())


def convert_to_lists(df):
    for i, row in tqdm(df.iterrows()):
        try:
            row['W_scoreLead'] = [float(x) for x in row['W_scoreLead'].split()]
            row['B_scoreLead'] = [float(x) for x in row['B_scoreLead'].split()]
            row['W_scoreSelfplay'] = [float(x) for x in row['W_scoreSelfplay'].split()]
            row['B_scoreSelfplay'] = [float(x) for x in row['B_scoreSelfplay'].split()]
            row['W_scoreStdev'] = [float(x) for x in row['W_scoreStdev'].split()]
            row['B_scoreStdev'] = [float(x) for x in row['B_scoreStdev'].split()]
            row['W_utility'] = [float(x) for x in row['W_utility'].split()]
            row['B_utility'] = [float(x) for x in row['B_utility'].split()]
            row['W_winrate'] = [float(x) for x in row['W_winrate'].split()]
            row['B_winrate'] = [float(x) for x in row['B_winrate'].split()]
            df.loc[i] = row
        except Exception as e:
            print(e)
            df = df.drop(index=[i])          
    return df
    

class MovesInfo:
    def __init__(self, row, n_moves=None, player='W'):
        enemy = 'B' if player == 'W' else 'W'
        if player == 'B':
            moves_len = min(len(row['B_winrate']), len(row['W_winrate']) + 1)
        else:
            moves_len = min(len(row['W_winrate']), len(row['B_winrate']))

        if n_moves is None:
            start_ind = 0
        else:
            start_ind = max(moves_len - n_moves - 1, 0)

        end_ind = moves_len
        w_0 = []
        sc_0 = []
        ut_0 = []
        std_0 = []
        sf_0 = []
        if player == 'B':
            if start_ind == 0:
                w_0 = [0.475]
                sc_0 = [0.]
                ut_0 = [0.]
                std_0 = [19. - 0.07666535852999867 * 2]
                sf_0 = [0.]
            else:
                w_0 = [row[enemy + '_winrate'][start_ind - 1]]

                sc_0 = [row[enemy + '_scoreLead'][start_ind - 1]]
                ut_0 = [row[enemy + '_utility'][start_ind - 1]]
                std_0 = [row[enemy + '_scoreStdev'][start_ind - 1]]
                sf_0 = [row[enemy + '_scoreSelfplay'][start_ind - 1]]
        self.winrate_delta = np.array(row[player + '_winrate'][start_ind:end_ind]) - np.array(
            (w_0 + row[enemy + '_winrate'])[start_ind:end_ind])
        self.score_delta = np.array(row[player + '_scoreLead'][start_ind:end_ind]) - np.array(
            (sc_0 + row[enemy + '_scoreLead'])[start_ind:end_ind])
        self.utility_delta = np.array(row[player + '_utility'][start_ind:end_ind]) - np.array(
            (ut_0 + row[enemy + '_utility'])[start_ind:end_ind])
        self.selfplay_delta = np.array(row[player + '_scoreSelfplay'][start_ind:end_ind]) - np.array(
            (sf_0 + row[enemy + '_scoreSelfplay'])[start_ind:end_ind])
        self.stddev_delta = np.array(row[player + '_scoreStdev'][start_ind:end_ind]) - np.array(
            (std_0 + row[enemy + '_scoreStdev'])[start_ind:end_ind])

        if player == 'B':
            self.winrate_delta = -self.winrate_delta
            self.score_delta = -self.score_delta
            self.utility_delta = -self.utility_delta
            self.selfplay_delta = -self.selfplay_delta

        self.move = row[player + '_move'].split()
        self.cnt_moves = end_ind - start_ind


def add_all_game_stats(df, player = 'W'):
    df['winrate_mean'] = None
    df['score_midmean'] = None
    df['score_mean'] = None
    df['score_var'] = None
    df['winrate_beauty_percent'] = None
    df['score_beauty_percent'] = None
    df['utility_mean'] = None
    df['score25p'] = None
    df['score75p'] = None
    df['winrate25p'] = None
    df['winrate75p'] = None
    df['winrate_midmean'] = None
    df['score50p'] = None
    df['selfplay_mean'] = None
    df['stddev_mean'] = None
    df['stddev50p'] = None
    df['stddev_last'] = None
    df['score_max'] = None
    df['score_min'] = None
    df['score_five_best_mean'] = None
    df['score_five_worst_mean'] = None
    for i, row in tqdm(df.iterrows()):
        add_basic_stats(row, MovesInfo(row, player=player))
        add_advanced_stats(row, MovesInfo(row, player=player))
        df.loc[i] = row
    return df            


def get_start_of_yose(margin_moves, no_change_count=5):
    '''
    Находим сколько последних ходов мы будем считать что это йосе

    Будем идти с конца по массиву количества ходов на краю, если в массиве no_change_count одинаковых чисел,
    то мы останавливаемся и говорим, что тут началось йосе
    '''
    no_change = 1
    ans = len(margin_moves)
    for i in range(len(margin_moves) - 2, -1, -1):
        if margin_moves[i] == margin_moves[i + 1]:
            no_change += 1
        else:
            no_change = 1
        if no_change >= no_change_count:
            ans = len(margin_moves) - i - 1
            break
    return ans


def reset_basic_stats(df, pref):
    df[pref + 'winrate_mean'] = None
    df[pref + 'winrate_beauty_percent'] = None
    df[pref + 'score_beauty_percent'] = None
    df[pref + 'score_mean'] = None
    df[pref + 'selfplay_mean'] = None
    df[pref + 'score_var'] = None
    df[pref + 'utility_mean'] = None


def is_marginal_move(move):
    return (move[0] == 'a' or move[0] == 'b') or (move[0] == 'r' or move[0] == 's') or \
           (move[1] == 'a' or move[1] == 'b') or (move[1] == 'r' or move[1] == 's')


def count_of_marginal_moves(moves):
    ans = np.zeros(len(moves))
    for i in range(len(moves)):
        ans[i] = is_marginal_move(moves[i])
    ans = np.cumsum(ans)
    return ans


def add_yose_stats(df, player = 'W'):
    pref = 'yose_'
    reset_basic_stats(df, pref)
    df['yose_length'] = None
    df['yose_start'] = None
    df['yose_has'] = None
    for i, row in tqdm(df.iterrows()):
        marginal_moves = count_of_marginal_moves(row['W_move'].split())
        n_moves = get_start_of_yose(marginal_moves, 10)
        add_basic_stats(row, MovesInfo(row, n_moves, player=player), pref)
        row['yose_length'] = n_moves
        row['yose_start'] = len(row['W_move'].split()) - n_moves
        row['yose_has'] = row['yose_start'] != 0
        df.loc[i] = row
    return df    


def delta_moves(a, b):
    return abs(ord(a[0]) - ord(b[0])) + abs(ord(a[1]) - ord(b[1]))


def get_distance_of_moves(moves):
    ans = np.zeros(len(moves) - 1)
    for i in range(1, len(moves)):
        ans[i - 1] = delta_moves(moves[i], moves[i - 1])
    return ans


def get_distance_from_enemy(my_moves, enemy_moves):
    ans = np.zeros(min(len(my_moves), len(enemy_moves)))
    for i in range(min(len(my_moves), len(enemy_moves))):
        ans[i] = delta_moves(my_moves[i], enemy_moves[i])
    return ans


def add_last_moves_stats(df, n_moves, pref=None, player='W'):
    if pref is None:
        pref = str(n_moves) + "_"
    reset_basic_stats(df, pref)
    for i, row in tqdm(df.iterrows()):
        add_basic_stats(row, MovesInfo(row, n_moves, player=player), pref)
        df.loc[i] = row
        
    return df    


def add_dist_stats_to_row(row, player='W'):
    dist = get_distance_of_moves(row[player+'_move'].split())
    dist_enemy = get_distance_from_enemy(row['W_move'].split(), row['B_move'].split())
    dist.sort()

    row['dist_mean'] = np.mean(dist)
    row['dist_var'] = np.var(dist)
    row['dist_median'] = dist[len(dist) // 2]
    row['dist_percent_more_than_10'] = np.mean([x > 10 for x in dist])
    row['dist_percent_more_than_5'] = np.mean([x > 5 for x in dist])
    row['dist_percent_more_than_20'] = np.mean([x > 20 for x in dist])

    row['dist_from_enemy_mean'] = np.mean(dist_enemy)
    row['dist_from_enemy_var'] = np.var(dist_enemy)


def add_dist_stats(df, player='W'):
    df['dist_mean'] = None
    df['dist_var'] = None
    df['dist_median'] = None
    df['dist_percent_more_than_5'] = None
    df['dist_percent_more_than_10'] = None
    df['dist_percent_more_than_20'] = None

    df['dist_from_enemy_mean'] = None
    df['dist_from_enemy_var'] = None

    for i, row in tqdm(df.iterrows()):
       if len(row['W_move']) > 20:
        add_dist_stats_to_row(row, player=player)
        df.loc[i] = row
    return df    


def delete_non_scalar_parameters(df):
    df.drop(['W_rating', 'B_rating', 'W_move', 'B_move', 'W_scoreLead', 'B_scoreLead', 'W_scoreSelfplay',
             'B_scoreSelfplay', 'W_scoreStdev', 'B_scoreStdev', 'W_utility', 'B_utility',
             'W_winrate', 'B_winrate', 'Result'], axis=1, inplace=True)
    return df         


def add_delta_lists_to_row(row, moves, player='W'):
    row['winrate'] = moves.winrate_delta
    row['score'] = moves.score_delta
    row['winrate_sqr'] = np.array([x ** 2 for x  in moves.winrate_delta])
    row['score_sqr'] = np.array([x ** 2 for x  in moves.score_delta])
    row['utility'] = moves.utility_delta
    row['selfplay'] = moves.selfplay_delta
    row['stddev'] = moves.stddev_delta
    row['dist_from_prev'] = get_distance_of_moves(moves.move)
    row['dist_more_5'] = [int(x > 5) + 1 for x in row['dist_from_prev']]
    enemy = 'B' if player == 'W' else 'W'
    row['dist_enemy'] = get_distance_from_enemy(row[player + '_move'].split(), row[enemy + '_move'].split())


def kWorst(moves, cnt=5):
    moves.sort()
    return moves[:cnt][-1]  


def addWorstMoves(df, player='W'):
  for cnt in [5, 10]:
    for type_cls in ['score', 'winrate']:
      df['Worst_' + type_cls + str(cnt)] = np.array(df.apply(lambda row: kWorst(row[type_cls], cnt), axis = 1))
  return df    


def add_lists_to_df(df, player='W'):
    df['winrate'] = None
    df['score'] = None
    df['utility'] = None
    df['selfplay'] = None
    df['stddev'] = None
    df['dist_from_prev'] = None
    df['score_sqr'] = None
    df['winrate_sqr'] = None
    df['dist_more_5'] = None
    df['dist_enemy'] = None
    for i, row in tqdm(df.iterrows()):
        add_delta_lists_to_row(row, MovesInfo(row, player=player), player)
        df.loc[i] = row
    return df    


def add_meta_to_df(df, player='W'):
    df['game_length'] = None
    df['rank'] = None
    df['color'] = None
    df['game_result'] = None
    for i, row in tqdm(df.iterrows()):
        try:
            add_meta(row, player)
            df.loc[i] = row
        except Exception as e:
            print(e)
            df = df.drop(index=[i])
    return df        


def get_feature_df(df, player='W'):
    df = add_meta_to_df(df, player)
    df = convert_to_lists(df)
    df = add_all_game_stats(df, player)
    df = add_yose_stats(df, player)
    df = add_last_moves_stats(df, 10, player)
    df = add_last_moves_stats(df, 20, player)
    df = add_dist_stats(df, player)
    df = delete_non_scalar_parameters(df)
    return df


def get_df_with_lists(df, player='W'):
    df = add_meta_to_df(df, player)
    df = convert_to_lists(df)
    df = add_lists_to_df(df, player)
    return df


In [23]:
from google.colab import drive
drive.mount('/content/drive')
katago_hum = pd.read_csv('/content/drive/MyDrive/data_big.csv')[:3000]
katago = pd.read_csv('/content/drive/MyDrive/computer_data.csv')
katago_hum['is_bot'] = 0
katago['is_bot'] = 1
#katago.dropna(how = 'any', inplace= True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
katago_1 = katago.append(katago_hum, ignore_index=True)

In [26]:
bots = ['noob_bot_1', 'doge_bot_2', 'kata-bot', 'katago-micro', 'doge_bot_3', 'doge_bot_1', 'amybot-ddk', 'doge_bot_4', '15bTurboLeela', '12bTurboSai', 'kata_noob']
katago_white = get_df_with_lists(katago_1.copy(), player = 'W')
katago_black = get_df_with_lists(katago_1.copy(), player = 'B')
katago_all = katago_white.append(katago_black, ignore_index=True)

12it [00:00, 57.64it/s]

'float' object has no attribute 'split'
'float' object has no attribute 'split'
'float' object has no attribute 'split'


36it [00:00, 57.49it/s]

'float' object has no attribute 'split'


72it [00:01, 55.49it/s]

...

2495it [00:28, 93.30it/s]

'float' object has no attribute 'split'


2536it [00:28, 97.71it/s]

'float' object has no attribute 'split'


5537it [01:03, 86.87it/s]
5472it [01:06, 82.60it/s]
5472it [01:31, 60.05it/s]


In [27]:
#katago_all = katago_all[features + ['is_bot']].dropna()
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(katago_all, katago_all['is_bot'], test_size=0.2)


In [28]:
katago_white.columns

Index(['W_rating', 'B_rating', 'W_nickname', 'B_nickname', 'Result', 'W_move',
       'B_move', 'W_scoreLead', 'B_scoreLead', 'W_scoreSelfplay',
       'B_scoreSelfplay', 'W_scoreStdev', 'B_scoreStdev', 'W_utility',
       'B_utility', 'W_winrate', 'B_winrate', 'game_id', 'error', 'is_bot',
       'W_visits', 'B_visits', 'game_length', 'rank', 'color', 'game_result',
       'winrate', 'score', 'utility', 'selfplay', 'stddev', 'dist_from_prev',
       'score_sqr', 'winrate_sqr', 'dist_more_5', 'dist_enemy'],
      dtype='object')

In [37]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
import pandas as pd


def get_tensor(df, features, sequence_len):
    def resize_to_length(m, length):
        if len(m) > length:
            return m[:length]
        n_nules = length - len(m)
        return list([0] * n_nules) + list(m)

    arr = []
    for feature in features:
        arr.append([np.array(y) for y in df[feature].apply(lambda x: resize_to_length(x, sequence_len))])
    arr = np.array(arr)
    arr = arr.swapaxes(0, 1)
    arr = arr.swapaxes(1, 2)
    return tf.convert_to_tensor(arr, np.float32)


class BotClassificatorRunner:
    __DEFAULT_FEATURES = ['score', 'winrate', 'utility', 'stddev', 'selfplay', 'dist_from_prev', 'winrate_sqr', 'score_sqr', 'dist_more_5', 'dist_enemy']

    def __create_rnn(self, hidden_units, input_shape):
        model = keras.Sequential()
        model.add(tf.keras.layers.Masking(mask_value=0.,
                                          input_shape=input_shape))
        model.add(layers.BatchNormalization())
        model.add(layers.Bidirectional(layers.LSTM(hidden_units)))  # , dropout = 0.15, reccurent_dropout = 0.15)))
        model.add(layers.Dense(units=2, activation="linear"))
        model.add(layers.Activation('softmax'))
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics='accuracy')
        return model

    def __init__(self, model=None, sequence_len=150, features=__DEFAULT_FEATURES):
        self.features = features
        self.sequence_len = sequence_len
        if model is None:
            self.model = self.__create_rnn(64, (sequence_len, len(features)))
        else:
            self.model = model

    def fit(self, df, target, val_X, val_y, epochs=10, batch_size=256):
        val_data = None
        if val_X is not None:
            val_y = keras.utils.to_categorical(tf.convert_to_tensor(np.array(val_y, dtype="int32"),
                                                                    np.int32), num_classes=2)
            val_X = get_tensor(val_X, self.features, self.sequence_len)
            val_data = (val_X, val_y)
        X = get_tensor(df, self.features, self.sequence_len)
        y = keras.utils.to_categorical(tf.convert_to_tensor(np.array(target, dtype="int32"), np.int32),
                                       num_classes=2)
        self.model.fit(X, y, validation_data=val_data, epochs=epochs, batch_size=batch_size, class_weight = {0:1, 1: 1.1})

    def get_probs(self, df):
        X = get_tensor(df, self.features, self.sequence_len)
        return self.model.predict(X)

    def predict(self, df):
        y = self.get_probs(df)
        return y.argmax(axis=1)


In [30]:
np.mean(y_test)

0.45043398812243035

In [38]:
rnn_features = ['score', 'winrate', 'utility', 'stddev', 'selfplay', 'dist_from_prev', 'winrate_sqr', 'score_sqr', 'dist_more_5', 'dist_enemy', 'W_scoreLead', 
                'B_scoreLead', 'W_scoreSelfplay',
       'B_scoreSelfplay', 'W_scoreStdev', 'B_scoreStdev', 'W_utility',
       'B_utility', 'W_winrate', 'B_winrate']
model = BotClassificatorRunner(sequence_len=150)

In [39]:
model.fit(X_train, y_train, X_test, y_test, epochs=25, batch_size=32)

Epoch 1/25
274/274 [==============================] - 63s 196ms/step - loss: 0.6011 - accuracy: 0.7142 - val_loss: 0.5409 - val_accuracy: 0.7474
Epoch 2/25
274/274 [==============================] - 52s 191ms/step - loss: 0.4914 - accuracy: 0.8016 - val_loss: 0.4058 - val_accuracy: 0.8360
Epoch 3/25
274/274 [==============================] - 51s 185ms/step - loss: 0.3789 - accuracy: 0.8623 - val_loss: 0.3047 - val_accuracy: 0.8803
Epoch 4/25
274/274 [==============================] - 51s 186ms/step - loss: 0.2923 - accuracy: 0.8910 - val_loss: 0.2342 - val_accuracy: 0.9114
Epoch 5/25
274/274 [==============================] - 50s 184ms/step - loss: 0.2534 - accuracy: 0.9004 - val_loss: 0.2227 - val_accuracy: 0.9068
Epoch 6/25
274/274 [==============================] - 51s 185ms/step - loss: 0.2283 - accuracy: 0.9102 - val_loss: 0.2041 - val_accuracy: 0.9146
Epoch 7/25
274/274 [==============================] - 51s 186ms/step - loss: 0.2136 - accuracy: 0.9146 - val_loss: 0.1998 - val_ac

In [36]:
model.model.save('/content/drive/MyDrive/has_botv1.h5')

In [ ]:
ans = model.predict(katago_all)

In [ ]:
np.mean(ans)